In [1]:
import mysql.connector
from datetime import datetime
import mysql.connector
import pandas as pd
import warnings
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import time
import numpy as np 
import matplotlib.gridspec as gridspec
from matplotlib.widgets import Button
import matplotlib.dates as mdates
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from gym import spaces

In [2]:
def get_table_names(host, user, password, database):
    try:
        # Establish a database connection
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Create a cursor object
        cursor = conn.cursor()

        # Execute a query to retrieve table names
        query = f"SELECT table_name FROM INFORMATION_SCHEMA.TABLES WHERE table_schema = '{database}'"
        cursor.execute(query)

        # Fetch all the rows
        tables = cursor.fetchall()

        # Print table names
        # for (table_name,) in tables:
        #     print(table_name)

    except mysql.connector.Error as error:
        print(f"Error: {error}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            return tables
        

In [3]:
def table_to_dataframe(host, user, password, database, table_name):
    try:
        # Establish a database connection
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Use backticks around the table name in the query
        query = f"SELECT * FROM `{table_name}`"

        # Use pandas to read the query into a DataFrame
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            df = pd.read_sql(query, conn)

        return df

    except mysql.connector.Error as error:
        print(f"Error: {error}")
        return None
    finally:
        if conn.is_connected():
            conn.close()
            


In [4]:
databaseA = 'a20240112'
password = '93150lbm!!'

tables = get_table_names('127.0.0.1', 'root', password, databaseA)

tables = [code for (code,) in tables]
print(tables)


['000440', '005290', '009620', '009730', '013990', '021080', '024060', '025320', '027360', '027830', '029480', '033540', '036540', '036630', '038870', '041020', '041190', '041460', '047310', '048770', '049080', '050890', '053050', '056080', '057680', '058610', '059100', '060250', '060280', '060310', '064260', '065350', '065450', '066790', '068760', '072950', '080220', '080530', '085670', '086960', '088800', '090150', '090360', '090710', '094170', '094480', '096630', '098460', '099440', '104480', '108490', '115160', '117730', '129890', '136480', '142210', '203650', '205100', '205470', '207760', '208340', '214270', '214680', '215100', '218150', '227100', '228760', '232140', '241820', '253590', '254120', '264850', '270520', '270660', '277070', '290690', '291230', '293580', '299900', '306620', '307930', '317830', '319400', '321260', '321370', '348340', '356680', '380540', '382800', '383930', '388790', '391710', '403870', '417200', '419050', '419270', '450520', '452190', '452300', '900250']

In [33]:
windowsize = 30
windwosize_2 = 5

code = tables[6]

df = table_to_dataframe('127.0.0.1', 'root', password, databaseA, code)

df['체결시간'] = pd.to_datetime(df['체결시간'])
df['현재가'] = pd.to_numeric(df['현재가'].str.replace('+','').str.replace('-',''), errors='coerce')
df['거래량'] = pd.to_numeric(df['거래량'], errors='coerce')
df['거래대금'] = df['현재가'] * df['거래량']
df = df.sort_values(by='체결시간', ignore_index=True)
df['time_diff_seconds'] = df['체결시간'].diff().dt.total_seconds()
df['time_diff_mean'] = df['time_diff_seconds'].rolling(window=windowsize).mean()
df['exp_minus'] = np.exp(-df['time_diff_seconds'])
df['signal'] = df['exp_minus'] * df['거래대금']
df['signal_mean5'] = df['signal'].rolling(window=windwosize_2).mean()

# df = df[['체결시간','time_diff_seconds','거래대금','signal_mean5']]

# # df.to_csv(f'{databaseA}_{code}.csv', index=False)
# df.to_csv(f'{databaseA}_{code}.csv', index=False, encoding='utf-8-sig')

print(code)
df




024060


,id,체결시간,현재가,거래량,거래대금,time_diff_seconds,time_diff_mean,exp_minus,signal,signal_mean5
0,3,2024-01-12 09:41:32.600788,11880,-18,-213840,NaN,NaN,NaN,NaN,NaN
1,6,2024-01-12 09:41:32.757373,11880,-875,-10395000,0.156585,NaN,8.550588e-01,-8.888337e+06,NaN
2,7,2024-01-12 09:41:32.806488,11880,-83,-986040,0.049115,NaN,9.520716e-01,-9.387807e+05,NaN
3,12,2024-01-12 09:41:32.910937,11910,80,952800,0.104449,NaN,9.008207e-01,8.583020e+05,NaN
4,16,2024-01-12 09:41:33.006461,11890,-1,-11890,0.095524,NaN,9.088965e-01,-1.080678e+04,NaN
...,...,...,...,...,...,...,...,...,...,...
136276,136277,2024-01-12 15:42:20.263866,12740,200,2548000,2.299887,44.738774,1.002702e-01,2.554884e+05,222528.964437
136277,136278,2024-01-12 15:42:34.568669,12740,20,254800,14.304803,45.215601,6.130600e-07,1.562077e-01,51302.233514
136278,136279,2024-01-12 15:42:38.361683,12740,5,63700,3.793014,45.340317,2.252760e-02,1.435008e+03,51398.391389
136279,136280,2024-01-12 15:42:43.264068,12740,4,50960,4.902385,45.502485,7.428844e-03,3.785739e+02,51469.883534


In [34]:
df_use = df[['체결시간','현재가']]
df_use

,체결시간,현재가
0,2024-01-12 09:41:32.600788,11880
1,2024-01-12 09:41:32.757373,11880
2,2024-01-12 09:41:32.806488,11880
3,2024-01-12 09:41:32.910937,11910
4,2024-01-12 09:41:33.006461,11890
...,...,...
136276,2024-01-12 15:42:20.263866,12740
136277,2024-01-12 15:42:34.568669,12740
136278,2024-01-12 15:42:38.361683,12740
136279,2024-01-12 15:42:43.264068,12740


In [35]:
df_use['delta'] = df_use['현재가'].diff(1)
df_use['up'] = df_use['delta'].clip(lower=0)
df_use['down'] =-1 * df_use['delta'].clip(upper=0)

df_use

C:\Users\pari0\AppData\Local\Temp\ipykernel_21012\2254305893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['delta'] = df_use['현재가'].diff(1)
C:\Users\pari0\AppData\Local\Temp\ipykernel_21012\2254305893.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['up'] = df_use['delta'].clip(lower=0)
C:\Users\pari0\AppData\Local\Temp\ipykernel_21012\2254305893.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

,체결시간,현재가,delta,up,down
0,2024-01-12 09:41:32.600788,11880,NaN,NaN,NaN
1,2024-01-12 09:41:32.757373,11880,0.0,0.0,-0.0
2,2024-01-12 09:41:32.806488,11880,0.0,0.0,-0.0
3,2024-01-12 09:41:32.910937,11910,30.0,30.0,-0.0
4,2024-01-12 09:41:33.006461,11890,-20.0,0.0,20.0
...,...,...,...,...,...
136276,2024-01-12 15:42:20.263866,12740,0.0,0.0,-0.0
136277,2024-01-12 15:42:34.568669,12740,0.0,0.0,-0.0
136278,2024-01-12 15:42:38.361683,12740,0.0,0.0,-0.0
136279,2024-01-12 15:42:43.264068,12740,0.0,0.0,-0.0


In [22]:
current_time = df_use['체결시간'][0] + pd.Timedelta(minutes=3)



period1 = current_time  - pd.Timedelta(seconds=15)
period2 = current_time  - pd.Timedelta(minutes=1)
period3 = current_time  - pd.Timedelta(minutes=3)
print(current_time)
print(period1)
print(period2)
print(period3)

avg_gain1 = df_use.loc[(period1 < df_use['체결시간']) & (df_use['체결시간'] < current_time), 'up'].mean()
avg_gain2 = df_use.loc[(period2 < df_use['체결시간']) & (df_use['체결시간'] < current_time), 'up'].mean()
avg_gain3 = df_use.loc[(period3 < df_use['체결시간']) & (df_use['체결시간'] < current_time), 'up'].mean()

avg_loss1 = df_use.loc[(period1 < df_use['체결시간']) & (df_use['체결시간'] < current_time), 'down'].mean()
avg_loss2 = df_use.loc[(period2 < df_use['체결시간']) & (df_use['체결시간'] < current_time), 'down'].mean()
avg_loss3 = df_use.loc[(period3 < df_use['체결시간']) & (df_use['체결시간'] < current_time), 'down'].mean()


rs1 = avg_gain1 / avg_loss1
rs2 = avg_gain2 / avg_loss2
rs3 = avg_gain3 / avg_loss3

# Calculate the RSI
rsi1 = 100 - (100 / (1 + rs1))
rsi2 = 100 - (100 / (1 + rs2))
rsi3 = 100 - (100 / (1 + rs3))

# Attach the RSI to the original data
df_use['RSI_1'] = rsi1
df_use['RSI_2'] = rsi2
df_use['RSI_3'] = rsi3


df_use

2024-01-12 09:44:32.600788
2024-01-12 09:44:17.600788
2024-01-12 09:43:32.600788
2024-01-12 09:41:32.600788


C:\Users\pari0\AppData\Local\Temp\ipykernel_21012\3239919422.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['RSI_1'] = rsi1
C:\Users\pari0\AppData\Local\Temp\ipykernel_21012\3239919422.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['RSI_2'] = rsi2
C:\Users\pari0\AppData\Local\Temp\ipykernel_21012\3239919422.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,체결시간,현재가,delta,up,down,RSI_1,RSI_2,RSI_3
0,2024-01-12 09:41:32.600788,11880,NaN,NaN,NaN,55.405405,54.545455,51.587302
1,2024-01-12 09:41:32.757373,11880,0.0,0.0,-0.0,55.405405,54.545455,51.587302
2,2024-01-12 09:41:32.806488,11880,0.0,0.0,-0.0,55.405405,54.545455,51.587302
3,2024-01-12 09:41:32.910937,11910,30.0,30.0,-0.0,55.405405,54.545455,51.587302
4,2024-01-12 09:41:33.006461,11890,-20.0,0.0,20.0,55.405405,54.545455,51.587302
...,...,...,...,...,...,...,...,...
136276,2024-01-12 15:42:20.263866,12740,0.0,0.0,-0.0,55.405405,54.545455,51.587302
136277,2024-01-12 15:42:34.568669,12740,0.0,0.0,-0.0,55.405405,54.545455,51.587302
136278,2024-01-12 15:42:38.361683,12740,0.0,0.0,-0.0,55.405405,54.545455,51.587302
136279,2024-01-12 15:42:43.264068,12740,0.0,0.0,-0.0,55.405405,54.545455,51.587302


In [36]:
df_use['delta'] = df_use['현재가'].diff(1)
df_use['up'] = df_use['delta'].clip(lower=0)
df_use['down'] =-1 * df_use['delta'].clip(upper=0)

df_use['RSI_1'] = None
df_use['RSI_2'] = None
df_use['RSI_3'] = None

for i in tqdm(range(len(df_use)),total=len(df_use)):

    current_time = df_use['체결시간'][i] 
    
    period1 = current_time  - pd.Timedelta(seconds=30)
    period2 = current_time  - pd.Timedelta(minutes=2)
    period3 = current_time  - pd.Timedelta(minutes=5)

    
    for period, rsi_col in zip([period1, period2, period3], ['RSI_1', 'RSI_2', 'RSI_3']):
        # Mask for the period
        mask = (df_use['체결시간'] > period) & (df_use['체결시간'] <= current_time)
        
        # Calculating average gains and losses
        avg_gain = df_use.loc[mask, 'up'].mean()
        avg_loss = df_use.loc[mask, 'down'].mean()

        # Handling division by zero
        if avg_loss == 0 and avg_gain == 0:
            rsi = np.nan  # Undefined when both gains and losses are 0
        elif avg_loss == 0:
            rsi = 100  # Theoretically 100 if only gains and no losses
        else:
            rs = avg_gain / avg_loss
            rsi = 100 - (100 / (1 + rs))

        # Assign the RSI value to the appropriate column
        df_use.at[i, rsi_col] = rsi


    
df_use 

C:\Users\pari0\AppData\Local\Temp\ipykernel_21012\8630571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['delta'] = df_use['현재가'].diff(1)
C:\Users\pari0\AppData\Local\Temp\ipykernel_21012\8630571.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['up'] = df_use['delta'].clip(lower=0)
C:\Users\pari0\AppData\Local\Temp\ipykernel_21012\8630571.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

  0%|          | 0/136281 [00:00<?, ?it/s]

,체결시간,현재가,delta,up,down,RSI_1,RSI_2,RSI_3
0,2024-01-12 09:41:32.600788,11880,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-12 09:41:32.757373,11880,0.0,0.0,-0.0,NaN,NaN,NaN
2,2024-01-12 09:41:32.806488,11880,0.0,0.0,-0.0,NaN,NaN,NaN
3,2024-01-12 09:41:32.910937,11910,30.0,30.0,-0.0,100,100,100
4,2024-01-12 09:41:33.006461,11890,-20.0,0.0,20.0,60.0,60.0,60.0
...,...,...,...,...,...,...,...,...
136276,2024-01-12 15:42:20.263866,12740,0.0,0.0,-0.0,NaN,NaN,NaN
136277,2024-01-12 15:42:34.568669,12740,0.0,0.0,-0.0,NaN,NaN,NaN
136278,2024-01-12 15:42:38.361683,12740,0.0,0.0,-0.0,NaN,NaN,NaN
136279,2024-01-12 15:42:43.264068,12740,0.0,0.0,-0.0,NaN,NaN,NaN


In [32]:
df_use.to_csv(f'{databaseA}_{code}.csv', index=True, encoding='utf-8-sig')
